In [238]:
import numpy as np
import pandas as pd

# score 함수 정의

In [239]:
# get rmsle
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

In [240]:
def get_ECLO(x1, x2, x3, x4):
    return x1*10+x2*5+x3*3+x4

# 전처리

In [241]:
train_df = pd.read_csv('../EDA/dataset/train.csv', index_col="ID")
test_df = pd.read_csv('../EDA/dataset/test.csv', index_col="ID")

In [242]:
label = train_df["ECLO"]

In [243]:
train_df.drop(["사망자수", "중상자수", "경상자수", "부상자수", "ECLO"], axis=1, inplace=True)

In [244]:
exclude_columns = train_df.columns.drop(["사고일시", "요일", "기상상태", "도로형태", "노면상태", "사고유형"])
train_df.drop(exclude_columns, axis=1, inplace=True)

In [245]:
def date_split(dataset):
    dataset["month"] = dataset["사고일시"].apply(lambda x: int(x.split("-")[1]))
    dataset["day"] = dataset["사고일시"].apply(lambda x: int(x.split(" ")[1]))
    return dataset
train_df = date_split(train_df)
test_df = date_split(test_df)
train_df.drop(["사고일시"], axis=1, inplace=True)
test_df.drop(["사고일시"], axis=1, inplace=True)

In [246]:
test_df.drop("시군구", axis=1, inplace=True)
test_df.head()

,요일,기상상태,도로형태,노면상태,사고유형,month,day
ID,,,,,,,
ACCIDENT_39609,토요일,맑음,교차로 - 교차로안,건조,차대사람,1,1
ACCIDENT_39610,토요일,맑음,단일로 - 기타,건조,차대사람,1,1
ACCIDENT_39611,토요일,맑음,교차로 - 교차로안,건조,차대차,1,4
ACCIDENT_39612,토요일,맑음,단일로 - 기타,건조,차대차,1,4
ACCIDENT_39613,토요일,맑음,교차로 - 교차로안,건조,차대차,1,6


In [247]:
train_df.head()

,요일,기상상태,도로형태,노면상태,사고유형,month,day
ID,,,,,,,
ACCIDENT_00000,화요일,맑음,단일로 - 기타,건조,차대사람,1,0
ACCIDENT_00001,화요일,흐림,단일로 - 기타,건조,차대사람,1,0
ACCIDENT_00002,화요일,맑음,단일로 - 기타,건조,차대사람,1,1
ACCIDENT_00003,화요일,맑음,단일로 - 기타,건조,차대차,1,2
ACCIDENT_00004,화요일,맑음,단일로 - 기타,건조,차대차,1,4


# model

In [248]:
# 카테고리 변수 트리 모델용으로 변환
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
train_df = ohe.fit_transform(train_df)
test_df = ohe.transform(test_df)
train_df

<39609x69 sparse matrix of type '<class 'numpy.float64'>'
	with 277263 stored elements in Compressed Sparse Row format>

In [249]:
# model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
rf = RandomForestRegressor(random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(train_df, label, test_size=0.2, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)
rmsle(y_valid, y_pred)

0.5060046278370087

In [252]:
# submission = rf.predict(test_df)
# test_df = pd.read_csv('../EDA/dataset/test.csv', index_col="ID")
# submission = pd.DataFrame(submission, index=test_df.index, columns=["ECLO"])
# submission.to_csv("submission.csv", index=True)